# 문제1

### **문제 1) Tokenizer 생성하기**

**1-1. `preprocessing()`**

텍스트 전처리를 하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장을 토큰화한 결과로, nested list 형태입니다. ex) [['i', 'go', 'to', 'school'], ['i', 'like', 'pizza']]
- 조건 1: 입력된 문장에 대해서 소문자로의 변환과 특수문자 제거를 수행합니다.
- 조건 2: 토큰화는 white space 단위로 수행합니다.
    
    

**1-2. `fit()`**

어휘 사전을 구축하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: 위에서 만든 `preprocessing` 함수를 이용하여 각 문장에 대해 토큰화를 수행합니다.
- 조건 2: 각각의 토큰을 정수 인덱싱 하기 위한 어휘 사전(`self.word_dict`)을 생성합니다.
    - 주어진 코드에 있는 `self.word_dict`를 활용합니다.
    

**1-3. `transform()`**

어휘 사전을 활용하여 입력 문장을 정수 인덱싱하는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장의 정수 인덱싱으로, nested list 형태입니다. ex) [[1, 2, 3, 4], [1, 5, 6]]
- 조건 1: 어휘 사전(`self.word_dict`)에 없는 단어는 'oov'의 index로 변환합니다.

In [1]:
import os
import string
import math
import pandas as pd

# 모듈 선언

cwd = os.getcwd()

with open(f'{cwd}/random_sentences.txt') as f:
    test_set = f.read().splitlines() 
docs = test_set[:5]

In [2]:
class Tokenizer():
  def __init__(self):
    self.word_dict = {'oov': 0}
    self.fit_checker = False
  
  def preprocessing(self, sequences):
    result = []
    for s in sequences:
        new_s = s.lower().translate(str.maketrans('', '', string.punctuation))
        result.append(list(new_s.split(' ')))
    return result
  
  def fit(self, sequences):
    self.fit_checker = False
    arr = self.preprocessing(sequences)
    fitted_docs = list(set(sum(arr, [])))
    token_index = 1
    for word in fitted_docs:
        self.word_dict[word] = token_index
        token_index+=1  
    self.fit_checker = True
    print(f"fitted word dict: {self.word_dict}")
  
  def transform(self, sequences):
    result = []
    fitted_docs = self.preprocessing(sequences)
    if self.fit_checker:
        for doc in fitted_docs:
            arr = [self.word_dict[w]  if self.word_dict[w] != None else self.word_dict['oov'] for w in doc]
            result.append(arr)
        return result
    else:
      raise Exception("Tokenizer instance is not fitted yet.")
      
  def fit_transform(self, sequences):
    self.fit(sequences)
    result = self.transform(sequences)
    return result




In [3]:
example_set = ['I go to school.', 'I LIKE pizza!']

token = Tokenizer()

print("문제 1-1")
arr = token.preprocessing(example_set)
print(f"전처리 결과값: {arr}")

print("문제 1-2")
token.fit(example_set)

result = token.transform(example_set)
print("문제 1-3")
print(f"결과값: {result}")

문제 1-1
전처리 결과값: [['i', 'go', 'to', 'school'], ['i', 'like', 'pizza']]
문제 1-2
fitted word dict: {'oov': 0, 'i': 1, 'to': 2, 'pizza': 3, 'go': 4, 'school': 5, 'like': 6}
문제 1-3
결과값: [[1, 4, 2, 5], [1, 6, 3]]


# 문제2

### **문제 2) TfidfVectorizer 생성하기**

**2-1. `fit()`**

입력 문장들을 이용해 IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: IDF 행렬은 list 형태입니다.
    - ex) [토큰1에 대한 IDF 값, 토큰2에 대한 IDF 값, .... ]
- 조건 2: IDF 값은 아래 식을 이용해 구합니다.
    
    $$
    idf(d,t)=log_e(\frac{n}{1+df(d,t)})
    $$
    
    - $df(d,t)$ : 단어 t가 포함된 문장 d의 개수
    - $n$ : 입력된 전체 문장 개수
- 조건 3: 입력된 문장의 토큰화에는 문제 1에서 만든 Tokenizer를 사용합니다.
    
    

**2-2. `transform()`**

입력 문장들을 이용해 TF-IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output : nested list 형태입니다.
    
    ex) [[tf-idf(1, 1), tf-idf(1, 2), tf-idf(1, 3)], [tf-idf(2, 1), tf-idf(2, 2), tf-idf(2, 3)]]
    
    |  | 토큰1 | 토큰2 | 토큰3 |
    | --- | --- | --- | --- |
    | 문장1 | tf-idf(1,1) | tf-idf(1,2) | tf-idf(1,3) |
    | 문장2 | tf-idf(2,1) | tf-idf(2,2) | tf-idf(2,3) |
- 조건1 : 입력 문장을 이용해 TF 행렬을 만드세요.
    - $tf(d, t)$ : 문장 d에 단어 t가 나타난 횟수
- 조건2 : 문제 2-1( `fit()`)에서 만든 IDF 행렬과 아래 식을 이용해 TF-IDF 행렬을 만드세요
    
    $$
    tf-idf(d,t) = tf(d,t) \times idf(d,t)
    $$

In [16]:
class TfidfVectorizer:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.fit_checker = False
  
    def fit(self, sequences):
        tokenized = self.tokenizer.fit_transform(sequences)
        idf_dict = {}
        for token in tokenized:
            for t in token:
                for word, value in self.tokenizer.word_dict.items():
                    if t == value:
                        idf_dict[word] = idf_dict.get(word, 0) + 1
                    else:
                        continue
        print(idf_dict)  
            
        
        self.fit_checker = True
    

    def transform(self, sequences):
        tf_dict = {}
        if self.fit_checker:
            tokenized = self.tokenizer.transform(sequences)
             
            return self.tfidf_matrix
        else:
            raise Exception("TfidfVectorizer instance is not fitted yet.")

  
    def fit_transform(self, sequences):
        self.fit(sequences)
        return self.transform(sequences)
 


In [17]:
tokenizer = Tokenizer()
tf_idf = TfidfVectorizer(tokenizer)
tf_idf.fit(docs)



fitted word dict: {'oov': 0, 'to': 1, 'ever': 2, 'sound': 3, 'ideas': 4, 'me': 5, 'list': 6, 'experience': 7, 'knowing': 8, 'on': 9, 'saw': 10, 'wanting': 11, 'suit': 12, 'other': 13, 'like': 14, 'onto': 15, 'is': 16, 'it': 17, 'his': 18, 'the': 19, 'awaited': 20, 'gingerly': 21, 'business': 22, 'clown': 23, 'she': 24, 'accept': 25, 'stepped': 26, 'will': 27, 'enchantment': 28, 'was': 29, 'wore': 30, 'irony': 31, 'travel': 32, 'neardeath': 33, 'no': 34, 'all': 35, 'asking': 36, 'he': 37, 'be': 38, 'her': 39, 'brought': 40, 'light': 41, 'when': 42, 'who': 43, 'bridge': 44, 'for': 45, 'doesnt': 46, 'my': 47, 'side': 48, 'new': 49, 'but': 50, 'change': 51, 'that': 52}
{'she': 2, 'saw': 1, 'no': 1, 'irony': 1, 'asking': 1, 'me': 2, 'to': 3, 'change': 1, 'but': 1, 'wanting': 1, 'accept': 1, 'her': 1, 'for': 1, 'who': 1, 'is': 1, 'he': 3, 'was': 1, 'all': 1, 'business': 1, 'when': 1, 'wore': 1, 'his': 1, 'clown': 1, 'suit': 1, 'stepped': 1, 'gingerly': 1, 'onto': 1, 'the': 3, 'bridge': 1, 'k

# IDF 구현하기

DF는 문서빈도 Document Frequency
이 값의 역수를 IDF라고 한다.

In [21]:
def get_word_list(docs):
    result = []
    for doc in docs:
        result += list(set(doc.lower().translate(str.maketrans('', '', string.punctuation)).split(' ')))
    result = set(result)
    return sorted(list(result));

def get_idf(t, docs):
    n = len(docs)
    df = sum([1 for doc in docs if t in doc])
    return math.log(n / (1 + df))
# pythonic
# return math.log(len(docs) / (1 + sum([1 for doc in docs if t in doc])))

word_list = get_word_list(docs)
idf_table = []

for word in word_list:
    value = get_idf(word, docs)
    idf_table.append(value)

idf_table_pd = pd.DataFrame(idf_table, index = word_list, columns = ["IDF"])
print(idf_table_pd)

                  IDF
accept       0.916291
all          0.916291
asking       0.916291
awaited      0.916291
be           0.916291
bridge       0.916291
brought      0.916291
business     0.916291
but          0.916291
change       0.916291
clown        0.916291
doesnt       1.609438
enchantment  0.916291
ever         0.916291
experience   0.916291
for          0.916291
gingerly     0.916291
he           0.000000
her          0.510826
his          0.916291
ideas        0.916291
irony        0.916291
is           0.223144
it           0.510826
knowing      0.916291
light        0.916291
like         0.916291
list         0.916291
me           0.510826
my           0.916291
neardeath    1.609438
new          0.916291
no           0.510826
on           0.223144
onto         0.916291
other        0.916291
saw          0.916291
she          0.916291
side         0.916291
sound        0.916291
stepped      0.916291
suit         0.916291
that         0.510826
the          0.916291
to        

## TF 구현하기
2-2 framsform()의 
- 조건1 : 입력 문장을 이용해 TF 행렬을 만드세요.
    - $tf(d, t)$ : 문장 d에 단어 t가 나타난 횟수
    
를 구해보겠습니다

In [56]:
tf_table = []

for doc in docs:
    row = []
    for word in word_list:
        row.append(doc.count(word))
    else:
        tf_table.append(row)

tf_table_result = pd.DataFrame(tf_table, columns = word_list)
print(tf_table_result)

   accept  all  asking  awaited  be  bridge  brought  business  but  change  \
0       1    0       1        0   0       0        0         0    1       1   
1       0    1       0        0   0       0        0         1    0       0   
2       0    0       0        1   0       1        0         0    0       0   
3       0    0       0        0   1       0        0         0    0       0   
4       0    0       0        0   0       0        1         0    0       0   

   ...  that  the  to  travel  wanting  was  when  who  will  wore  
0  ...     0    0   2       0        1    0     0    1     0     0  
1  ...     0    0   0       0        0    1     1    0     0     1  
2  ...     1    3   1       0        0    0     0    0     0     0  
3  ...     1    0   0       1        0    0     0    0     1     0  
4  ...     0    0   1       0        0    0     0    0     0     0  

[5 rows x 52 columns]


# 문제풀이

fitted word dict: {'oov': 0, 'other': 1, 'her': 2, 'doesnt': 3, 'wanting': 4, 'it': 5, 'but': 6, 'wore': 7, 'side': 8, 'experience': 9, 'ever': 10, 'my': 11, 'light': 12, 'who': 13, 'list': 14, 'she': 15, 'his': 16, 'is': 17, 'was': 18, 'like': 19, 'accept': 20, 'sound': 21, 'business': 22, 'change': 23, 'to': 24, 'the': 25, 'no': 26, 'saw': 27, 'bridge': 28, 'for': 29, 'will': 30, 'be': 31, 'knowing': 32, 'me': 33, 'irony': 34, 'ideas': 35, 'he': 36, 'brought': 37, 'neardeath': 38, 'that': 39, 'asking': 40, 'enchantment': 41, 'suit': 42, 'awaited': 43, 'on': 44, 'new': 45, 'onto': 46, 'clown': 47, 'all': 48, 'stepped': 49, 'gingerly': 50, 'when': 51, 'travel': 52}
[[15, 27, 26, 34, 40, 33, 24, 23, 6, 4, 33, 24, 20, 2, 29, 13, 15, 17], [36, 18, 48, 22, 51, 36, 7, 16, 47, 42], [36, 49, 50, 46, 25, 28, 32, 39, 41, 43, 44, 25, 1, 8], [5, 3, 21, 19, 39, 30, 10, 31, 44, 11, 52, 14], [25, 38, 9, 37, 45, 35, 24, 12]]
oov 0


TypeError: 'in <string>' requires string as left operand, not int